# Review NLP Helpfulness Prediction TFIDF (EBooks)

## Creating Spark Session & Importing All Necessary Libraries

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from sparknlp.base import *
from sparknlp.annotator import *

import sparknlp

# Start SparkSession with Spark NLP
# start() functions has 5 parameters: gpu, spark23, spark24, spark32, and memory
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(spark23=True) is when you have Apache Spark 2.3.x installed
# sparknlp.start(spark24=True) is when you have Apache Spark 2.4.x installed
# sparknlp.start(spark32=True) is when you have Apache Spark 3.2.x installed
# sparknlp.start(memory="16G") to change the default driver memory in SparkSession
spark = sparknlp.start(gpu = True)

In [3]:
from pyspark.sql.functions import lower, col
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import DataFrame
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier, NaiveBayes
import numpy as np

import nltk
import matplotlib.pyplot as plt
%matplotlib inline

#nltk.download('wordnet')

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kenne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Reading in Data

In [5]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv(r"C:\Users\kenne\OneDrive\Desktop\UChicago\Python\Final Project Big Data Amazon Review\Amazon Review Kaggle Data\amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv",inferSchema=True,header=True, sep='\t' )

## Code Cleaning

In [6]:
df.show(1, vertical = True, truncate = False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------
 marketplace       | US                                                                                                                               
 customer_id       | 33605939                                                                                                                         
 review_id         | RGYFDX8QXKEIR                                                                                                                    
 product_id        | B007KO2MLO                                                                                                                       
 product_parent    | 328837464                                                                                                                        
 product_title     | Big Maria                                                                

In [7]:
df = df.select('product_id', 'star_rating', 'product_category', 'review_headline', 'review_body', 'helpful_votes', 'total_votes')

In [8]:
df.show(1, vertical = True, truncate = False)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------
 product_id       | B007KO2MLO                                                                                                                       
 star_rating      | 4                                                                                                                                
 product_category | Digital_Ebook_Purchase                                                                                                           
 review_headline  | Quirky                                                                                                                           
 review_body      | Elmore Leonard meets the cast of Sierra Madre. Just a quirky read that will make you want to keep trying no matter what happens. 
 helpful_votes    | 0                                                                               

In [9]:
df.dropna().count()

5101525

In [10]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|product_id|star_rating|product_category|review_headline|review_body|helpful_votes|total_votes|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|         0|         17|              12|             63|        125|           17|         17|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+



In [11]:
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.count()
df2.show(5, vertical = True)

-RECORD 0----------------
 product_id       | 2    
 star_rating      | 17   
 product_category | 12   
 review_headline  | 5041 
 review_body      | 9632 
 helpful_votes    | 17   
 total_votes      | 17   



In [12]:
df = df.filter(col('total_votes') > 10)
df.count()

196219

In [13]:
df = df.fillna("", "review_body")
df = df.fillna("", "review_headline")

In [14]:
df = df.withColumn('review_text', F.concat('review_headline', F.lit(" "), 'review_body'))
df.show(1, vertical = True, truncate = False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 product_id       | B006NG0GXQ                                                                                                                                                

In [15]:
df.filter((col("review_text").isNull()) | ( col("review_text") == "")).show(1, vertical = True, truncate = False)

(0 rows)



In [16]:
df.select([count(when((col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show(5, vertical = True)

-RECORD 0---------------
 product_id       | 0   
 star_rating      | 0   
 product_category | 0   
 review_headline  | 2   
 review_body      | 4   
 helpful_votes    | 0   
 total_votes      | 0   
 review_text      | 0   



In [17]:
df = df.withColumn('helpful_ratio', F.col('helpful_votes') / F.col('total_votes'))
df.show(1, vertical = True, truncate = False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 product_id       | B006NG0GXQ                                                                                                                                                

In [18]:
df.filter(col('helpful_ratio') < 0).count()

0

In [19]:
df = df.withColumn('helpful', when(col("helpful_ratio") < 0.5, 0).otherwise(1))

In [20]:
df.show(5, vertical = True)

-RECORD 0--------------------------------
 product_id       | B006NG0GXQ           
 star_rating      | 1                    
 product_category | Digital_Ebook_Pur... 
 review_headline  | Completely absurd    
 review_body      | From the time tha... 
 helpful_votes    | 36                   
 total_votes      | 47                   
 review_text      | Completely absurd... 
 helpful_ratio    | 0.7659574468085106   
 helpful          | 1                    
-RECORD 1--------------------------------
 product_id       | B0078T6YHY           
 star_rating      | 1                    
 product_category | Digital_Ebook_Pur... 
 review_headline  | No thanks            
 review_body      | This work should ... 
 helpful_votes    | 16                   
 total_votes      | 26                   
 review_text      | No thanks This wo... 
 helpful_ratio    | 0.6153846153846154   
 helpful          | 1                    
-RECORD 2--------------------------------
 product_id       | B00CKXCNH8    

### Cleaning the Review_Text

In [21]:
df_clean = df

In [22]:
df_clean.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- review_headline: string (nullable = false)
 |-- review_body: string (nullable = false)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_text: string (nullable = false)
 |-- helpful_ratio: double (nullable = true)
 |-- helpful: integer (nullable = false)



In [23]:
df_clean.select("helpful").distinct().show()

+-------+
|helpful|
+-------+
|      1|
|      0|
+-------+



In [24]:
#Making the review text to all lower case
df_clean=df_clean.withColumn('review_text_l', F.lower(F.col('review_text')))

In [25]:
df_clean.select(df_clean.columns[10]).show(5, vertical = True)

-RECORD 0-----------------------------
 review_text_l | completely absurd... 
-RECORD 1-----------------------------
 review_text_l | no thanks this wo... 
-RECORD 2-----------------------------
 review_text_l | fell in love with... 
-RECORD 3-----------------------------
 review_text_l | a big disappointm... 
-RECORD 4-----------------------------
 review_text_l | holy hotness, bla... 
only showing top 5 rows



In [26]:
#Removing all of the additional punctuations
df_clean=df_clean.withColumn('review_text_l', F.regexp_replace('review_text_l', '\'', ''))
                                                                 
df_clean=df_clean.withColumn('review_text_l', F.regexp_replace('review_text_l', '\\n|[^\w]', ' ')).withColumn('review_text_l', F.regexp_replace('review_text_l', '\s+', ' '))

In [27]:
df_clean.select('review_text_l').show(5,vertical = True, truncate = False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
#Removing reviews that have 3 or less words
df_clean.filter(F.length(df_clean.review_text_l) < 3).count()

0

In [29]:
df_clean = df_clean.filter(F.length(df_clean.review_text_l) > 3)

In [30]:
df_clean.count()

196219

## NLP Pipeline

This pipeline consists of the following: Tokenizer, Stop Words Remover, Stemming, & TFIDF

In [31]:
#Document & Tokenize
document_assembler = DocumentAssembler().setInputCol("review_text_l").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("review_words")
 
#Cleaning Tokens
remover           = StopWordsCleaner().setInputCols("review_words").setOutputCol("review_words_stop").setCaseSensitive(False).setStopWords(eng_stopwords)
#lemmatizer        = Lemmatizer().setInputCols(["review_words_stop"]).setOutputCol("review_words_lemstem")
stemmer           = Stemmer().setInputCols(["review_words_stop"]).setOutputCol("review_words_lemstem")
finisher          = Finisher().setInputCols(["review_words_lemstem"]).setOutputCols(["token_features"]).setOutputAsArray(True).setCleanAnnotations(False)
#hashingTF         = HashingTF(inputCol="token_features", outputCol="rawFeatures")
#idf               = IDF(inputCol="rawFeatures", outputCol="features")

pipeline_stem = Pipeline(stages=[document_assembler,tokenizer,remover,stemmer,finisher])#,hashingTF,idf])   

##### Running NLP pipeline

In [32]:
%%time
df_clean_nlp = pipeline_stem.fit(df_clean).transform(df_clean)

Wall time: 319 ms


In [33]:
df_clean_nlp.show(1)

+----------+-----------+--------------------+-----------------+--------------------+-------------+-----------+--------------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|star_rating|    product_category|  review_headline|         review_body|helpful_votes|total_votes|         review_text|     helpful_ratio|helpful|       review_text_l|            document|        review_words|   review_words_stop|review_words_lemstem|      token_features|
+----------+-----------+--------------------+-----------------+--------------------+-------------+-----------+--------------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B006NG0GXQ|          1|Digital_Ebook_Pur...|Completely absurd|From the time tha...|           36|         47|Completely absurd...|0.765957446808

##### Hashing TF

In [34]:
hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures = 10000)
df_featurizedData = hashingTF.transform(df_clean_nlp)
df_featurizedData.show(1)

+----------+-----------+--------------------+-----------------+--------------------+-------------+-----------+--------------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|star_rating|    product_category|  review_headline|         review_body|helpful_votes|total_votes|         review_text|     helpful_ratio|helpful|       review_text_l|            document|        review_words|   review_words_stop|review_words_lemstem|      token_features|         rawFeatures|
+----------+-----------+--------------------+-----------------+--------------------+-------------+-----------+--------------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B006NG0GXQ|          1|Digital_Ebook_Pur...|Completely absurd|From the time tha..

##### IDF

In [35]:
%%time
idf = IDF(inputCol="rawFeatures", outputCol="features")
df_nlp = idf.fit(df_featurizedData).transform(df_featurizedData)
df_nlp.show(1, vertical = True, truncate = False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
df_nlp.show(1, vertical = True, truncate = False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
nlpdf_model = df_nlp.select('helpful','features')

In [38]:
from pyspark.sql.types import IntegerType
nlpdf_model = nlpdf_model.withColumn('helpful',col('helpful').cast(IntegerType()))

##### Train & Test Split

In [39]:
#Sampling only 10% for modeling
nlpdf_model = nlpdf_model.sample(False, 0.1, seed=0)

In [40]:
train, test = nlpdf_model.randomSplit([0.8, 0.2], seed=12345)

## NLP Modeling

#### Logistic Regression (1 = Helpful, 0 = Not Helpful)

In [41]:
%%time
lr = LogisticRegression(featuresCol = 'features', labelCol='helpful')

paramGrid = (ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.0]).addGrid(lr.elasticNetParam, [0.0]).addGrid(lr.maxIter, [20, 100]).build())

#Evaluator
evaluator = MulticlassClassificationEvaluator(labelCol='helpful', predictionCol="prediction")
    
# Create 3-fold CrossValidator
cv = CrossValidator(estimator=lr,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)

cvModel = cv.fit(train)

predictions = cvModel.transform(train)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.9783265804874911
0.9783189700579743
0.9783119150071177
0.9783265804874912
Wall time: 2min 21s


In [42]:
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])

{Param(parent='LogisticRegression_e6498c04e62e', name='regParam', doc='regularization parameter (>= 0).'): 0.0, Param(parent='LogisticRegression_e6498c04e62e', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_e6498c04e62e', name='maxIter', doc='max number of iterations (>= 0).'): 20}


In [43]:
%%time
predictions = cvModel.transform(test)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.7088220295767466
0.7138275314239957
0.7195543057575067
0.7088220295767466
Wall time: 28.5 s


#### Naive Bayes Classifier (1 = Helpful, 0 = Not Helpful)

In [44]:
%%time
nb = NaiveBayes(featuresCol='features', labelCol='helpful')

paramGrid = (ParamGridBuilder().addGrid(nb.smoothing, [0.1, 0.5, 1.0]).build())

#Evaluator
evaluator = MulticlassClassificationEvaluator(labelCol='helpful', predictionCol="prediction")
    
# Create 3-fold CrossValidator
cv = CrossValidator(estimator=nb,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)

cvModel = cv.fit(train)

predictions = cvModel.transform(train)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.8050678500225095
0.8163937093892284
0.8461896412197023
0.8050678500225095
Wall time: 1min 22s


In [45]:
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])

{Param(parent='NaiveBayes_4a7b3a01908d', name='smoothing', doc='The smoothing parameter, should be >= 0, default is 1.0'): 0.1}


In [46]:
%%time
predictions = cvModel.transform(test)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.7108618052014278
0.7235403241188446
0.7426694448015619
0.7108618052014278
Wall time: 28.5 s


#### Random Forest Classifier (1 = Helpful, 0 = Not Helpful)

In [47]:
%%time
rfc = RandomForestClassifier(impurity="gini", featuresCol='features', labelCol="helpful")

paramGrid = (ParamGridBuilder().addGrid(rfc.impurity, ['gini', 'entropy']).addGrid(rfc.maxBins, [32, 100]).build())

#Evaluator
evaluator = MulticlassClassificationEvaluator(labelCol='helpful', predictionCol="prediction")
    
# Create 3-fold CrossValidator
cv = CrossValidator(estimator=rfc,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)

cvModel = cv.fit(train)

predictions = cvModel.transform(train)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.7732330053379639
0.6744146988233282
0.8246595825940959
0.7732330053379638
Wall time: 2min 19s


In [48]:
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])

{Param(parent='RandomForestClassifier_4af7276ee4f0', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini', Param(parent='RandomForestClassifier_4af7276ee4f0', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32}


In [49]:
%%time
predictions = cvModel.transform(test)

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}))

0.7758796532381438
0.6779617472515928
0.6019892363089422
0.7758796532381438
Wall time: 28.3 s
